In [1]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


'File'
corpus/ResumeNER/test.char.bmes


Loading model cost 0.582 seconds.
Prefix dict has been built succesfully.


corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-18 16:02:31.957395
		Done!
Generating Dictionary of Token Unique...	 2019-06-18 16:02:32.018250
		The length of DTU is: 1891 	 2019-06-18 16:02:32.018646
Generating the ORIGTokenIndex...       	 2019-06-18 16:02:32.018697
		The idx of token is: 0 	 2019-06-18 16:02:32.018920
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/Token1891/Pyramid/CORPUS.p
CORPUS	the le

In [2]:
CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': True,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    'annoE':   {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
}


BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Deal with the Channel: token
Current Channel is        	 token
Current Channel Max_Ngram 	 1
Deal with the Channel: basic
Current Channel is        	 basic
Current Channel Max_Ngram 	 1
Deal with the Channel: medical
Current Channel is        	 medical
Current Channel Max_Ngram 	 1
Deal with the Channel: radical
Current Channel is        	 radical
Current Channel Max_Ngram 	 1
Deal with the Channel: subcomp
Current Channel is        	 subcomp
Current Channel Max_Ngram 	 1
Deal with the Channel: stroke
Current Channel is        	 stroke
Current Channel Max_Ngram 	 1
Deal with the Channel: pos
Current Channel is        	 pos
Current Channel Max_Ngram 	 1
Deal with the Channel: annoE
Current Channel is        	 annoE
Current Channel Max_Ngram 	 1


In [3]:
def get_sent_strfeats(sent, Channel_Settings, train = True):
    '''
        sent is a nlptex.sentence object
        return a pandas dataframe
    '''
    features = {}
    # stroke 12 and subcomp 6 are fixed internally.
    for ch, cs in Channel_Settings.items():
        # print(ch)
        if 'anno' in ch and not train:
            continue
        
        feature = sent.getChannelGrain(ch)
        # this will cost a lot of time
        if ch == 'stroke':
            max_leng = 12
            feature2 = []
            for token_feat in feature:
                if len(token_feat) <= max_leng:
                    feature2.append(token_feat + ['</'] * (max_leng - len(token_feat))) 
                else:
                    feature2.append(token_feat[:max_leng])
            feature = feature2
        elif ch == 'subcomp':
            max_leng = 6
            feature2 = []
            for token_feat in feature:
                if len(token_feat) <= max_leng:
                    feature2.append(token_feat + ['</'] * (max_leng - len(token_feat)))
                else:
                    feature2.append(token_feat[:max_leng])
            feature = feature2 
            
        
        features[ch] = feature
        # print(feature)
    L = []
    for ch, feat in features.items():
        L.append(pd.DataFrame(feat))
    
    Feats = pd.concat(L, axis = 1)
    return Feats


In [4]:
sents = corpus.Sentences#[:100]

In [6]:
fidx = 3
idx = 3
'U'  + str(fidx) + ':%x[0,' + str(idx) + ']'

'U3:%x[0,3]'

In [7]:
def generate_template(gram_1 = 20, path = '_tmp/template'):
    L = ['# Unigram\n\n']
    for idx in range(gram_1):
        L.append('U'  + str(idx) + ':%x[0,' + str(idx) + ']\n')
        
    L.append('\n\n')
    L.append('# Bigram\nB')
    with open(path, 'w') as f:
        f.write(''.join(L))
    return ''.join(L)


print(generate_template(gram_1 = 20))

# Unigram

U0:%x[0,0]
U1:%x[0,1]
U2:%x[0,2]
U3:%x[0,3]
U4:%x[0,4]
U5:%x[0,5]
U6:%x[0,6]
U7:%x[0,7]
U8:%x[0,8]
U9:%x[0,9]
U10:%x[0,10]
U11:%x[0,11]
U12:%x[0,12]
U13:%x[0,13]
U14:%x[0,14]
U15:%x[0,15]
U16:%x[0,16]
U17:%x[0,17]
U18:%x[0,18]
U19:%x[0,19]


# Bigram
B


In [8]:
import pandas as pd
import numpy as np
def generate_template(gram_1 = 20, path = '_tmp/template'):
    L = ['# Unigram\n\n']
    for idx in range(gram_1):
        L.append('U'  + str(idx) + ':%x[0,' + str(idx) + ']\n')
        
    L.append('\n\n')
    L.append('# Bigram\nB')
    with open(path, 'w') as f:
        f.write(''.join(L))
    return ''.join(L)

def crfpp_train(sents, Channel_Settings, model_path, ):
    # create para
    feats_data_path = '_tmp/_crfpp_train_feats.txt'
    template_path   = '_tmp/_template'
    DFtrain = pd.DataFrame()
    for sent in sents:
        df = get_sent_strfeats(sent, Channel_Settings)
        df.loc[len(df)] = np.NaN     ## Trick Here
        DFtrain = DFtrain.append(df) ## Trick Here
    DFtrain = DFtrain.reset_index(drop=True)
    DFtrain.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )

    generate_template(gram_1 = DFtrain.shape[1] - 1, path = template_path)
    crf_learn(feats_data_path, model_path, template_path  = template_path)
    # return para 


In [12]:
model_path = 'models/model'
Channel_Settings = BasicObject.CHANNEL_SETTINGS

In [58]:
trainSents = sents[:int(len(sents) * 0.8)]
testSents  = sents[int(len(sents) * 0.8):]

In [59]:
crfpp_train(trainSents, Channel_Settings, model_path)

CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 3500.. 3600.. 
Done!0.49 s

Number of sentences: 3693
Number of features:  60299
Number of thread(s): 8
Freq:                1
eta:                 0.00010
C:                   1.00000
shrinking size:      20
iter=0 terr=0.99789 serr=1.00000 act=60299 obj=340606.07501 diff=1.00000
iter=1 terr=0.60943 serr=1.00000 act=60299 obj=203644.54032 diff=0.40211
iter=2 terr=0.48190 serr=1.00000 act=60299 obj=164479.76157 diff=0.19232
iter=3 terr=0.36063 serr=1.00000 act=60299 obj=144422.13705 diff=0.12195
iter=4 terr=0.35356 serr=1.00000 act=60299 obj=137855.05564 diff=0.04547
iter=5 terr=0.40784 serr=1.00000 act=60299 obj=118829.11026 diff=0.13801
iter=6 terr=

In [44]:
from crftools import read_result, extractSET, crf_test, match_anno_pred_result, calculate_F1_Score


def read_result(result_path):
    result = pd.read_csv(result_path, sep = '\t', header = None, skip_blank_lines=False)
    # get the last column of the result
    return result.iloc[:,-1].dropna().values


def tagger(sent, model_path, Channel_Settings = None):
    '''
        basically from crf_test
        sent: a sentence, could be without annotation
    '''
    # 1. get sentence feats
    # hopefully, the model_config is included in model_path

    feats_data_path   = '_tmp/_tagger_feats.txt'
    results_data_path = '_tmp/_tagger_results.txt'

    sentence_with_feats = sent
    df = get_sent_strfeats(sent, Channel_Settings, train = False)
    df.to_csv(feats_data_path, sep = '\t', encoding = 'utf=8', header = False, index = False )
    # 2. save the sentence feats to a file
    
    # 3. tag a sentence
    crf_test(feats_data_path, model_path, results_data_path)

    # 4. read and parse the result to pred_SSET
    # get a tag_seq
    # list of tuple (score, result)
    tag_seq = read_result(results_data_path)
    pred_SET = extractSET(tag_seq)
    return pred_SET


def get_sent_annoSET(sent, channel = 'annoE', tagScheme = 'BIOES'):
    anno_seq = [i[0] for i in sent.getChannelGrain(channel, tagScheme=tagScheme)]
    anno_SET = extractSET(anno_seq)
    return anno_SET

get_sent_annoSET(sent)

[(2, 17, 'ORG'), (17, 22, 'TITLE'), (23, 38, 'ORG'), (38, 42, 'TITLE')]

In [45]:
from nlptext.sentence import Sentence
sent = Sentence(10)
sent.sentence

'现任大股东无锡产业发展集团有限公司董事局董事、无锡威孚高科技集团股份有限公司党委书记。'

In [49]:
def load_anno(channel_anno, tagScheme, BasicObject):
    # channel_anno = 'annoE'
    GU = BasicObject.getGrainUnique(channel_anno, tagScheme = tagScheme)
    list_annoE = GU[0]
    tag_size = len(list_annoE)
    label_list = list(set([i.split('-')[0] for i in list_annoE if '-' in i]))
    label_list.sort()
    return channel_anno, tagScheme, tag_size, list_annoE, label_list

channel_anno = 'annoE'
tagScheme = 'BIOE'
channel_anno, tagScheme, tag_size, list_annoE, label_list = load_anno(channel_anno, tagScheme, BasicObject)

In [53]:
pred_SET = tagger(sent, model_path='models/model', Channel_Settings = Channel_Settings)
anno_SET = get_sent_annoSET(sent)


match_anno_pred_result(anno_SET, pred_SET, label_list = label_list)

,E_Anno,E_Pred,E_Match,CONT_Anno,CONT_Pred,CONT_Match,EDU_Anno,EDU_Pred,EDU_Match,LOC_Anno,...,ORG_Match,PRO_Anno,PRO_Pred,PRO_Match,RACE_Anno,RACE_Pred,RACE_Match,TITLE_Anno,TITLE_Pred,TITLE_Match
0,4,4,4,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,2,2,2


In [62]:
label_list

['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']

In [63]:
def crfpp_test(sents, model_path, Channel_Settings, label_list):
    '''
        sents: a list of sents
    '''
    label_list = [] # get the label list
    pred_entities = []
    anno_entities = []
    # here sents are a batch of sents, not necessary to be all the sents
    for sent in sents:
        pred_SET = tagger(sent, model_path, Channel_Settings)
        pred_entities.append(pred_SET)
        anno_SET = get_sent_annoSET(sent)
        anno_entities.append(anno_SET)
    
    # return anno_entities, pred_entities
    Result = match_anno_pred_result(anno_entities, pred_entities, label_list = label_list)
    # return Result
    R = calculate_F1_Score(Result, label_list)
    return R

anno_entities, pred_entities = crfpp_test(testSents, model_path, Channel_Settings, label_list)

In [77]:

Result = match_anno_pred_result(anno_entities, pred_entities, label_list = label_list)

Result

,E_Anno,E_Pred,E_Match,CONT_Anno,CONT_Pred,CONT_Match,EDU_Anno,EDU_Pred,EDU_Match,LOC_Anno,...,ORG_Match,PRO_Anno,PRO_Pred,PRO_Match,RACE_Anno,RACE_Pred,RACE_Match,TITLE_Anno,TITLE_Pred,TITLE_Match
0,2,2,2,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,4,2,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,3,2,1
2,2,2,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,1
3,4,4,4,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,3,3,3
4,3,3,3,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,1,1,1
5,15,15,14,0,0,0,0,0,0,0,...,5,0,0,0,0,0,0,9,9,9
6,5,5,5,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,3,3,3
7,4,4,4,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,2,2,2
8,2,2,2,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,1
9,3,3,3,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2,2,2


In [79]:

def calculate_F1_Score(Result, label_list):
    Result = Result.sum().to_dict()
    List = []
    entitiesLabel = label_list + ['E']
    # entitiesLabel = ['Sy','Bo', 'Ch', 'Tr', 'Si'] + ['R'] + ['E']
    for eL in entitiesLabel:
        d = dict()
        d['id'] = eL
        for k in Result:
            if eL == k.split('_')[0]:
                d[k.replace(eL + '_','')] = Result[k]
        List.append(d)
    
    R = pd.DataFrame(List)
    R.set_index('id', inplace = True)
    R.index.name = None
    # print(R)
    R['R'] = R['Match']/R['Anno']
    R['P'] = R['Match']/R['Pred']
    R['F1'] = 2*R['R']*R['P']/(R['R'] + R['P'])
    return R[['Anno', 'Pred', 'Match', 'R', 'P', 'F1']]
    # return R[['Anno', 'Pred', 'Match']]

R = calculate_F1_Score(Result, label_list) # use stroke, subcomp
R

,Anno,Pred,Match,R,P,F1
CONT,67,66,65,0.970149,0.984848,0.977444
EDU,213,219,207,0.971831,0.945205,0.958333
LOC,10,5,4,0.400000,0.800000,0.533333
NAME,215,223,194,0.902326,0.869955,0.885845
ORG,1131,1129,886,0.783378,0.784765,0.784071
PRO,50,48,38,0.760000,0.791667,0.775510
RACE,23,24,23,1.000000,0.958333,0.978723
TITLE,1526,1533,1316,0.862385,0.858447,0.860412
E,3235,3247,2733,0.844822,0.841700,0.843258


In [6]:
crfpp_test(testSents, Channel_Settings, model_path, label_list) # don't use stroke, subcomp

,Anno,Pred,Match,R,P,F1
CONT,67,68,66,0.985075,0.970588,0.977778
EDU,213,215,208,0.976526,0.967442,0.971963
LOC,10,6,5,0.500000,0.833333,0.625000
NAME,215,225,201,0.934884,0.893333,0.913636
ORG,1131,1119,898,0.793988,0.802502,0.798222
PRO,50,49,38,0.760000,0.775510,0.767677
RACE,23,24,23,1.000000,0.958333,0.978723
TITLE,1526,1531,1327,0.869594,0.866754,0.868171
E,3235,3237,2766,0.855023,0.854495,0.854759


In [11]:
crfpp_test(testSents, Channel_Settings, model_path, label_list)

,Anno,Pred,Match,R,P,F1
CONT,67,67,66,0.985075,0.985075,0.985075
EDU,213,215,206,0.967136,0.958140,0.962617
LOC,10,4,4,0.400000,1.000000,0.571429
NAME,215,229,205,0.953488,0.895197,0.923423
ORG,1131,1088,838,0.740937,0.770221,0.755295
PRO,50,47,35,0.700000,0.744681,0.721649
RACE,23,23,23,1.000000,1.000000,1.000000
TITLE,1526,1530,1276,0.836173,0.833987,0.835079
E,3235,3203,2653,0.820093,0.828286,0.824169


In [1]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    'annoE':   {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
}


BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.570 seconds.


'File'
corpus/ResumeNER/test.char.bmes


Prefix dict has been built succesfully.


corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-18 17:02:19.927576
		Done!
Generating Dictionary of Token Unique...	 2019-06-18 17:02:19.993215
		The length of DTU is: 1891 	 2019-06-18 17:02:19.993622
Generating the ORIGTokenIndex...       	 2019-06-18 17:02:19.993691
		The idx of token is: 0 	 2019-06-18 17:02:19.993857
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/Token1891/Pyramid/CORPUS.p
CORPUS	the le

In [2]:
model_path = 'models/model'
Channel_Settings = BasicObject.CHANNEL_SETTINGS
sents = corpus.Sentences#[:100]

In [3]:
trainSents = sents[:int(len(sents) * 0.8)]
testSents  = sents[int(len(sents) * 0.8):]

In [4]:
from crftools import load_anno, crfpp_train, crfpp_test

channel_anno = 'annoE'
tagScheme = 'BIOE'
channel_anno, tagScheme, tag_size, list_annoE, label_list = load_anno(channel_anno, tagScheme, BasicObject)

In [5]:
Channel_Settings

{'token': {'Max_Ngram': 1},
 'basic': {'Max_Ngram': 1, 'end_grain': False},
 'medical': {'Max_Ngram': 1, 'end_grain': False},
 'radical': {'Max_Ngram': 1, 'end_grain': False},
 'pos': {'Max_Ngram': 1, 'end_grain': False, 'tagScheme': 'BIOE'},
 'annoE': {'Max_Ngram': 1, 'end_grain': False, 'tagScheme': 'BIOE'}}

In [11]:
crfpp_train(trainSents, Channel_Settings, model_path)

CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 3500.. 3600.. 
Done!0.17 s

Number of sentences: 3693
Number of features:  37196
Number of thread(s): 8
Freq:                1
eta:                 0.00010
C:                   1.00000
shrinking size:      20
iter=0 terr=0.99789 serr=1.00000 act=37196 obj=340606.07501 diff=1.00000
iter=1 terr=0.52990 serr=1.00000 act=37196 obj=257554.26041 diff=0.24384
iter=2 terr=0.47448 serr=1.00000 act=37196 obj=253514.68197 diff=0.01568
iter=3 terr=0.65118 serr=1.00000 act=37196 obj=354099.48341 diff=0.39676
iter=4 terr=0.52151 serr=1.00000 act=37196 obj=220141.96642 diff=0.37830
iter=5 terr=0.52189 serr=1.00000 act=37196 obj=156063.11430 diff=0.29108
iter=6 terr=

In [8]:
crfpp_test(testSents, Channel_Settings, model_path, label_list)

,Anno,Pred,Match,R,P,F1
CONT,67,68,66,0.985075,0.970588,0.977778
EDU,213,215,208,0.976526,0.967442,0.971963
LOC,10,6,5,0.500000,0.833333,0.625000
NAME,215,225,201,0.934884,0.893333,0.913636
ORG,1131,1119,898,0.793988,0.802502,0.798222
PRO,50,49,38,0.760000,0.775510,0.767677
RACE,23,24,23,1.000000,0.958333,0.978723
TITLE,1526,1531,1327,0.869594,0.866754,0.868171
E,3235,3237,2766,0.855023,0.854495,0.854759


In [7]:
label_list

['CONT', 'EDU', 'LOC', 'NAME', 'ORG', 'PRO', 'RACE', 'TITLE']

In [9]:
from pprint import pprint
from nlptext.base import BasicObject

########### ResumeNER ###########
CORPUSPath = 'corpus/ResumeNER/'
corpusFileIden = '.bmes'
textType   = 'block'
Text2SentMethod  = 're'
Sent2TokenMethod = 'iter'
TOKENLevel = 'char'
anno = 'embed'
annoKW = {}

BasicObject.INIT(CORPUSPath, corpusFileIden, textType, 
                 Text2SentMethod, Sent2TokenMethod, TOKENLevel, 
                 anno, annoKW)

from nlptext.corpus import Corpus
corpus = Corpus()
# corpus.IdxFolderStartEnd

print([i for i in corpus.TOKEN])

CHANNEL_SETTINGS_TEMPLATE = {
    # CTX_IND
    'token':   {'use': True, 'Max_Ngram': 1,},
    'char':    {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'basic':   {'use': True,'Max_Ngram': 1, 'end_grain': False},
    'medical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'radical': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'subcomp': {'use': False,'Max_Ngram': 1, 'end_grain': False},
    'stroke':  {'use': False,'Max_Ngram': 1, 'end_grain': False},
    # CTX_DEP
    'pos':     {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    # ANNO
    'annoR':   {'use': False,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
    'annoE':   {'use': True,'Max_Ngram': 1, 'end_grain': False, 'tagScheme':   'BIOE',},
}


BasicObject.BUILD_GRAIN_UNI_AND_LOOKUP(CHANNEL_SETTINGS_TEMPLATE=CHANNEL_SETTINGS_TEMPLATE)


model_path = 'models/model'
Channel_Settings = BasicObject.CHANNEL_SETTINGS
sents = corpus.Sentences#[:100]

trainSents = sents[:int(len(sents) * 0.8)]
testSents  = sents[int(len(sents) * 0.8):]

from crftools import load_anno, crfpp_train, crfpp_test

channel_anno = 'annoE'
tagScheme = 'BIOE'
channel_anno, tagScheme, tag_size, list_annoE, label_list = load_anno(channel_anno, tagScheme, BasicObject)

'File'
corpus/ResumeNER/test.char.bmes
corpus/ResumeNER/train.char.bmes
corpus/ResumeNER/dev.char.bmes
Total Num of All    Tokens 149812
The Total Number of Tokens: 149812
Counting the number unique Tokens...          	 2019-06-18 17:05:57.029187
		Done!
Generating Dictionary of Token Unique...	 2019-06-18 17:05:57.077954
		The length of DTU is: 1891 	 2019-06-18 17:05:57.078393
Generating the ORIGTokenIndex...       	 2019-06-18 17:05:57.078445
		The idx of token is: 0 	 2019-06-18 17:05:57.078586
		Done!
Only Keep First 3500000 Tokens.
The coverage rate is: 0.0
Total Num of Unique Tokens 1891
['</pad>', '</start>', '</end>', 'O', 'CONT-B', 'CONT-E', 'CONT-I', 'CONT-S', 'EDU-B', 'EDU-E', 'EDU-I', 'EDU-S', 'LOC-B', 'LOC-E', 'LOC-I', 'LOC-S', 'NAME-B', 'NAME-E', 'NAME-I', 'NAME-S', 'ORG-B', 'ORG-E', 'ORG-I', 'ORG-S', 'PRO-B', 'PRO-E', 'PRO-I', 'PRO-S', 'RACE-B', 'RACE-E', 'RACE-I', 'RACE-S', 'TITLE-B', 'TITLE-E', 'TITLE-I', 'TITLE-S']
CORPUS	it is Dumped into file: data/ResumeNER/char/T

In [10]:
crfpp_train(trainSents, Channel_Settings, model_path)

CRF++: Yet Another CRF Tool Kit
Copyright (C) 2005-2013 Taku Kudo, All rights reserved.

reading training data: 100.. 200.. 300.. 400.. 500.. 600.. 700.. 800.. 900.. 1000.. 1100.. 1200.. 1300.. 1400.. 1500.. 1600.. 1700.. 1800.. 1900.. 2000.. 2100.. 2200.. 2300.. 2400.. 2500.. 2600.. 2700.. 2800.. 2900.. 3000.. 3100.. 3200.. 3300.. 3400.. 3500.. 3600.. 
Done!0.12 s

Number of sentences: 3693
Number of features:  30787
Number of thread(s): 8
Freq:                1
eta:                 0.00010
C:                   1.00000
shrinking size:      20
iter=0 terr=0.99789 serr=1.00000 act=30787 obj=340606.07501 diff=1.00000
iter=1 terr=0.60245 serr=1.00000 act=30787 obj=276240.68737 diff=0.18897
iter=2 terr=0.60245 serr=1.00000 act=30787 obj=201812.64032 diff=0.26943
iter=3 terr=0.65196 serr=1.00000 act=30787 obj=188995.72741 diff=0.06351
iter=4 terr=0.64514 serr=1.00000 act=30787 obj=134728.34953 diff=0.28714
iter=5 terr=0.49660 serr=1.00000 act=30787 obj=125767.44040 diff=0.06651
iter=6 terr=

In [12]:
crfpp_test(testSents, Channel_Settings, model_path, label_list)

,Anno,Pred,Match,R,P,F1
CONT,67,67,66,0.985075,0.985075,0.985075
EDU,213,215,206,0.967136,0.958140,0.962617
LOC,10,4,4,0.400000,1.000000,0.571429
NAME,215,229,205,0.953488,0.895197,0.923423
ORG,1131,1088,838,0.740937,0.770221,0.755295
PRO,50,47,35,0.700000,0.744681,0.721649
RACE,23,23,23,1.000000,1.000000,1.000000
TITLE,1526,1530,1276,0.836173,0.833987,0.835079
E,3235,3203,2653,0.820093,0.828286,0.824169
